In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
data = pd.read_csv("../data/clean.csv").set_index('Unnamed: 0')
data

,group,region,01D2Z36,0319-6G20,0319-7L14,27F-1492R,37-13,67-14,A2,Abiotrophia,...,Xanthomonas,Xenorhabdus,Xylanibacillus,Yersinia,Youngiibacter,Zavarzinia,Zoogloea,ZOR0006,Unclassified,study_site
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SRR4457864,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.640935,1
SRR4457873,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.630561,1
SRR4457876,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.448002,1
SRR4457877,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.459400,1
SRR4457878,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.031991,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR24874401,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.423813,14
SRR24874402,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.704500,14
SRR24874404,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.894488,14


In [20]:
# for every study, get the number of nonzero and zero features
for i in range(1, 15):
    study = data[data['study_site'] == i]
    print(f'For study {i}, there are {(study.sum() == 0).sum()} trivial features, and {(study.sum() != 0).sum()} non-trivial features.')

For study 1, there are 888 trivial features, and 243 non-trivial features.
For study 2, there are 1017 trivial features, and 114 non-trivial features.
For study 3, there are 858 trivial features, and 273 non-trivial features.
For study 4, there are 810 trivial features, and 321 non-trivial features.
For study 5, there are 772 trivial features, and 359 non-trivial features.
For study 6, there are 923 trivial features, and 208 non-trivial features.
For study 7, there are 931 trivial features, and 200 non-trivial features.
For study 8, there are 982 trivial features, and 149 non-trivial features.
For study 9, there are 887 trivial features, and 244 non-trivial features.
For study 10, there are 946 trivial features, and 185 non-trivial features.
For study 11, there are 516 trivial features, and 615 non-trivial features.
For study 12, there are 952 trivial features, and 179 non-trivial features.
For study 13, there are 988 trivial features, and 143 non-trivial features.
For study 14, there 

In [47]:
data_by_site = data.groupby('study_site').sum()
(data_by_site.iloc[:, 2:] != 0).sum().value_counts()

1     644
2     128
3      78
11     51
4      31
14     29
0      28
5      20
10     19
8      19
7      19
12     18
6      17
13     16
9      10
Name: count, dtype: int64

There are 28 features where no study contained nonzero values for its abundance. 

There are 644 features where only one study contained nonzero values for its abundance.

In [66]:
# filter out rare OTUs: < 0.01 (values are in percentages)
data_16s = data.drop(columns=['group', 'region', 'study_site'])
rare_OTU_threshold = 1
rare_OTUs = data_16s.columns[(data_16s <= 1).sum() == data_16s.shape[0]]

In [67]:
filter_rare = data_16s.drop(columns=rare_OTUs)
filter_rare

,Acetatifactor,Acetivibrio,Acidaminococcus,Acinetobacter,Actinomyces,Acutalibacter,Adlercreutzia,Aeromicrobium,Agathobacter,Agathobaculum,...,UCG-003,UCG-004,UCG-005,UCG-009,UCG-010,Varibaculum,Veillonella,Weissella,Xanthomonas,Unclassified
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SRR4457864,0.0,0.0,0.000000,0.0,0.026594,0.0,0.033847,0.0,1.716510,0.0,...,0.055605,0.0,0.834079,0.000000,0.000000,0.0,0.000000,0.0,0.0,3.640935
SRR4457873,0.0,0.0,0.000000,0.0,0.016925,0.0,0.000000,0.0,2.676499,0.0,...,0.198259,0.0,0.041103,0.000000,0.009671,0.0,0.043520,0.0,0.0,7.630561
SRR4457876,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.985162,0.0,...,0.192395,0.0,0.033523,0.000000,0.000000,0.0,0.119518,0.0,0.0,7.448002
SRR4457877,0.0,0.0,0.000000,0.0,0.019861,0.0,0.000000,0.0,2.467344,0.0,...,0.465969,0.0,0.200137,0.000000,0.000000,0.0,0.000000,0.0,0.0,6.459400
SRR4457878,0.0,0.0,0.000000,0.0,0.037431,0.0,0.034935,0.0,1.247692,0.0,...,0.416729,0.0,1.766732,0.064880,0.137246,0.0,0.000000,0.0,0.0,7.031991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR24874401,0.0,0.0,0.000000,0.0,0.262331,0.0,0.000000,0.0,0.000000,0.0,...,0.274705,0.0,0.009899,0.000000,0.000000,0.0,0.064345,0.0,0.0,25.423813
SRR24874402,0.0,0.0,0.000000,0.0,0.035169,0.0,0.000000,0.0,6.541490,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.189523,0.0,0.0,3.704500
SRR24874404,0.0,0.0,0.000000,0.0,0.474964,0.0,0.155638,0.0,0.614501,0.0,...,0.000000,0.0,0.037568,0.000000,0.000000,0.0,0.110020,0.0,0.0,1.894488


In [68]:
covariates = data[['group', 'region', 'study_site']]
covariates

,group,region,study_site
Unnamed: 0,,,
SRR4457864,0,0,1
SRR4457873,0,0,1
SRR4457876,0,0,1
SRR4457877,0,0,1
SRR4457878,0,0,1
...,...,...,...
SRR24874401,1,1,14
SRR24874402,1,1,14
SRR24874404,1,1,14


In [70]:
pd.concat([covariates, filter_rare], axis=1)

,group,region,study_site,Acetatifactor,Acetivibrio,Acidaminococcus,Acinetobacter,Actinomyces,Acutalibacter,Adlercreutzia,...,UCG-003,UCG-004,UCG-005,UCG-009,UCG-010,Varibaculum,Veillonella,Weissella,Xanthomonas,Unclassified
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
SRR4457864,0,0,1,0.0,0.0,0.000000,0.0,0.026594,0.0,0.033847,...,0.055605,0.0,0.834079,0.000000,0.000000,0.0,0.000000,0.0,0.0,3.640935
SRR4457873,0,0,1,0.0,0.0,0.000000,0.0,0.016925,0.0,0.000000,...,0.198259,0.0,0.041103,0.000000,0.009671,0.0,0.043520,0.0,0.0,7.630561
SRR4457876,0,0,1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.192395,0.0,0.033523,0.000000,0.000000,0.0,0.119518,0.0,0.0,7.448002
SRR4457877,0,0,1,0.0,0.0,0.000000,0.0,0.019861,0.0,0.000000,...,0.465969,0.0,0.200137,0.000000,0.000000,0.0,0.000000,0.0,0.0,6.459400
SRR4457878,0,0,1,0.0,0.0,0.000000,0.0,0.037431,0.0,0.034935,...,0.416729,0.0,1.766732,0.064880,0.137246,0.0,0.000000,0.0,0.0,7.031991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR24874401,1,1,14,0.0,0.0,0.000000,0.0,0.262331,0.0,0.000000,...,0.274705,0.0,0.009899,0.000000,0.000000,0.0,0.064345,0.0,0.0,25.423813
SRR24874402,1,1,14,0.0,0.0,0.000000,0.0,0.035169,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.189523,0.0,0.0,3.704500
SRR24874404,1,1,14,0.0,0.0,0.000000,0.0,0.474964,0.0,0.155638,...,0.000000,0.0,0.037568,0.000000,0.000000,0.0,0.110020,0.0,0.0,1.894488


In [71]:
sparcc = pd.read_csv('../src/hc_sparcc.csv')
sparcc

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,266,267,268,269,270,271,272,273,274,275
0,0,1.000000,0.206635,0.030753,-0.014439,-0.028667,0.020503,-0.008386,0.006252,0.025708,...,0.025239,-0.028284,-0.028983,-0.009959,-0.030108,-0.009504,0.011510,0.006342,-0.022108,0.034100
1,1,0.206635,1.000000,0.027362,0.008117,0.017461,0.008917,-0.001020,0.008811,0.007903,...,0.009213,0.012085,-0.032219,-0.016148,-0.018801,-0.037304,0.018278,0.006916,-0.011493,-0.008855
2,2,0.030753,0.027362,1.000000,0.025350,-0.002424,0.010966,0.007005,-0.009559,-0.013968,...,-0.008211,-0.025444,0.002754,-0.006989,-0.007682,-0.019072,-0.006343,0.010896,-0.026483,0.004243
3,3,-0.014439,0.008117,0.025350,1.000000,0.007453,-0.004441,-0.002297,0.005787,-0.009887,...,0.015959,-0.010462,-0.029196,-0.000607,-0.022652,0.001323,0.006255,-0.002179,0.003259,-0.043474
4,4,-0.028667,0.017461,-0.002424,0.007453,1.000000,-0.002617,-0.006397,0.012123,-0.008241,...,-0.016826,0.003636,0.010987,-0.010189,0.003605,-0.009335,0.011462,-0.022002,0.016134,-0.013577
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,271,-0.009504,-0.037304,-0.019072,0.001323,-0.009335,0.003617,-0.004969,0.007350,-0.000317,...,-0.020092,0.001314,0.007395,-0.011723,0.008077,1.000000,0.010679,-0.006638,0.002093,-0.035312
272,272,0.011510,0.018278,-0.006343,0.006255,0.011462,0.038823,0.016164,-0.021410,-0.019177,...,-0.011691,0.015514,-0.033350,-0.024375,-0.001164,0.010679,1.000000,0.008993,-0.002112,-0.049677
273,273,0.006342,0.006916,0.010896,-0.002179,-0.022002,-0.002223,-0.000594,-0.022770,-0.008500,...,0.006204,-0.009535,-0.003697,-0.017544,-0.001827,-0.006638,0.008993,1.000000,-0.013527,0.007023
274,274,-0.022108,-0.011493,-0.026483,0.003259,0.016134,0.037003,0.017535,-0.022827,0.001554,...,-0.006966,0.004881,0.035838,0.002699,0.000370,0.002093,-0.002112,-0.013527,1.000000,-0.010555
